In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv('data/fullDF.csv', index_col=0)
dataset = dataset.drop(['GRID_TYPE', 'PLAYER_ID', 'SHOT_ATTEMPTED_FLAG', 'TEAM_ID'], axis=1)
dataset_copy = dataset

In [3]:
categorical_cols = ['ACTION_TYPE', 'HTM', 'PERIOD', 'PLAYER_NAME', 'QUARTER', 'SHOT_TYPE', 'SHOT_ZONE_AREA',
                    'SHOT_ZONE_BASIC', 'SHOT_ZONE_RANGE', 'TEAM_NAME', 'VTM']

def onehot_encode(data):
    for feature in categorical_cols:
        one_hot = pd.get_dummies(data[feature], prefix=feature)
        data = data.drop(feature, axis = 1)
        data = data.join(one_hot)
    return data

dataset = onehot_encode(dataset)

In [4]:
correlation_df = dataset.corr()[['SHOT_MADE_FLAG']]
correlation_df.rename(columns={"SHOT_MADE_FLAG": "correlation_with_making_shot"}, inplace=True)
correlation_df

,correlation_with_making_shot
EVENTTIME,0.012396
LOC_X,0.007481
LOC_Y,-0.036743
MINUTES_REMAINING,0.011510
SECONDS_REMAINING,0.010951
...,...
VTM_SAC,0.004892
VTM_SAS,0.003040
VTM_TOR,-0.004145
VTM_UTA,0.000401


In [5]:
correlation_df = correlation_df.sort_values(by='correlation_with_making_shot', ascending=False)

In [6]:
for i, j in correlation_df.iterrows():
    corr = j[0]
    if corr > 0.03 or corr < -0.03:
        print(i, j) 
        print() 

SHOT_MADE_FLAG correlation_with_making_shot    1.0
Name: SHOT_MADE_FLAG, dtype: float64

SHOT_ZONE_BASIC_Restricted Area correlation_with_making_shot    0.199887
Name: SHOT_ZONE_BASIC_Restricted Area, dtype: float64

SHOT_ZONE_RANGE_Less Than 8 ft. correlation_with_making_shot    0.167723
Name: SHOT_ZONE_RANGE_Less Than 8 ft., dtype: float64

SHOT_ZONE_AREA_Center(C) correlation_with_making_shot    0.133036
Name: SHOT_ZONE_AREA_Center(C), dtype: float64

PLAYER_AVG correlation_with_making_shot    0.120922
Name: PLAYER_AVG, dtype: float64

SHOT_TYPE_2PT Field Goal correlation_with_making_shot    0.12018
Name: SHOT_TYPE_2PT Field Goal, dtype: float64

ACTION_TYPE_Dunk Shot correlation_with_making_shot    0.10812
Name: ACTION_TYPE_Dunk Shot, dtype: float64

ACTION_TYPE_Cutting Layup Shot correlation_with_making_shot    0.085503
Name: ACTION_TYPE_Cutting Layup Shot, dtype: float64

ACTION_TYPE_Cutting Dunk Shot correlation_with_making_shot    0.082773
Name: ACTION_TYPE_Cutting Dunk Shot, d

The most correlated features with the shot result are:

Shot Zone

Player Average

Shot Type

Action Type

Shot Distance

LOC_Y

In [7]:
dataset_imp_features_only = dataset_copy[['SHOT_MADE_FLAG', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE',
                                    'PLAYER_AVG', 'SHOT_TYPE', 'ACTION_TYPE', 'SHOT_DISTANCE', 'LOC_Y']]
dataset_imp_features_only

,SHOT_MADE_FLAG,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,PLAYER_AVG,SHOT_TYPE,ACTION_TYPE,SHOT_DISTANCE,LOC_Y
2732,0,Above the Break 3,Left Side Center(LC),24+ ft.,0.510288,3PT Field Goal,Jump Shot,25,125.2763
12130,1,Restricted Area,Center(C),Less Than 8 ft.,0.357466,2PT Field Goal,Running Dunk Shot,0,-5.5908
74191,1,Restricted Area,Center(C),Less Than 8 ft.,0.508380,2PT Field Goal,Hook Shot,2,78.5379
19621,0,Restricted Area,Center(C),Less Than 8 ft.,0.452555,2PT Field Goal,Layup Shot,1,238.1948
66474,0,Above the Break 3,Right Side Center(RC),24+ ft.,0.244444,3PT Field Goal,Jump Shot,25,139.8303
...,...,...,...,...,...,...,...,...,...
70944,0,Mid-Range,Right Side Center(RC),16-24 ft.,0.538462,2PT Field Goal,Jump Shot,18,217.1834
76301,0,Above the Break 3,Right Side Center(RC),24+ ft.,0.487013,3PT Field Goal,Running Pull-Up Jump Shot,25,206.2505
48761,0,In The Paint (Non-RA),Center(C),8-16 ft.,0.448649,2PT Field Goal,Jump Shot,12,246.5670
2308,0,Restricted Area,Center(C),Less Than 8 ft.,0.504292,2PT Field Goal,Layup Shot,1,50.1469


In [8]:
train_data, test_data = train_test_split(dataset, test_size=0.2)
x_train = train_data.drop(['SHOT_MADE_FLAG'], axis=1)
y_train = train_data[['SHOT_MADE_FLAG']]
x_test = test_data.drop(['SHOT_MADE_FLAG'], axis=1)
y_test = test_data[['SHOT_MADE_FLAG']]

categorical_cols1 = ['ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_AREA', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_RANGE']

def onehot_encode1(data):
    for feature in categorical_cols1:
        one_hot = pd.get_dummies(data[feature], prefix=feature)
        data = data.drop(feature, axis = 1)
        data = data.join(one_hot)
    return data

dataset_imp_features_only = onehot_encode1(dataset_imp_features_only)

train_data1, test_data1 = train_test_split(dataset_imp_features_only, test_size=0.2)
x_train1 = train_data1.drop(['SHOT_MADE_FLAG'], axis=1)
y_train1 = train_data1[['SHOT_MADE_FLAG']]
x_test1 = test_data1.drop(['SHOT_MADE_FLAG'], axis=1)
y_test1 = test_data1[['SHOT_MADE_FLAG']]

In [9]:
x_train.head()
x_train1.head()

,PLAYER_AVG,SHOT_DISTANCE,LOC_Y,ACTION_TYPE_Alley Oop Dunk Shot,ACTION_TYPE_Alley Oop Layup shot,ACTION_TYPE_Cutting Dunk Shot,ACTION_TYPE_Cutting Finger Roll Layup Shot,ACTION_TYPE_Cutting Layup Shot,ACTION_TYPE_Driving Bank Hook Shot,ACTION_TYPE_Driving Bank shot,...,SHOT_ZONE_BASIC_In The Paint (Non-RA),SHOT_ZONE_BASIC_Left Corner 3,SHOT_ZONE_BASIC_Mid-Range,SHOT_ZONE_BASIC_Restricted Area,SHOT_ZONE_BASIC_Right Corner 3,SHOT_ZONE_RANGE_16-24 ft.,SHOT_ZONE_RANGE_24+ ft.,SHOT_ZONE_RANGE_8-16 ft.,SHOT_ZONE_RANGE_Back Court Shot,SHOT_ZONE_RANGE_Less Than 8 ft.
54950,0.416667,25,359.1825,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
24403,0.402299,9,78.2657,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
49866,0.488789,1,17.3021,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
31536,0.362140,6,62.7118,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
33052,0.413534,2,163.8914,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


We tested 3 models - these models are:
1. 3-layer RNN using all of our features.
2. 3-layer RNN using only the most correlated features (calculated above)
3. 2-layer RNN using all of our features.

In [10]:
# 1. 3-layer RNN using all of our features.
model = Sequential()
model.add(Dense(1000, input_dim=len(x_train.columns), activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=80, batch_size=1024)

Epoch 1/80
67573/67573 [==============================] - 6s 82us/step - loss: 2.6374 - accuracy: 0.5395
Epoch 2/80
67573/67573 [==============================] - 4s 64us/step - loss: 0.6829 - accuracy: 0.5857
Epoch 3/80
67573/67573 [==============================] - 5s 77us/step - loss: 0.7345 - accuracy: 0.5731
Epoch 4/80
67573/67573 [==============================] - 5s 68us/step - loss: 0.6790 - accuracy: 0.5980
Epoch 5/80
67573/67573 [==============================] - 5s 68us/step - loss: 0.6701 - accuracy: 0.6081
Epoch 6/80
67573/67573 [==============================] - 5s 70us/step - loss: 0.6891 - accuracy: 0.5965
Epoch 7/80
67573/67573 [==============================] - 4s 65us/step - loss: 0.6623 - accuracy: 0.6184
Epoch 8/80
67573/67573 [==============================] - 4s 66us/step - loss: 0.6465 - accuracy: 0.6305
Epoch 9/80
67573/67573 [==============================] - 4s 65us/step - loss: 0.6651 - accuracy: 0.6170
Epoch 10/80
67573/67573 [==============================

In [11]:
y_test_pred = model.predict_classes(x_test)

In [14]:
print(f"accuracy: {round(accuracy_score(y_test, y_test_pred), 3)}")
print(f"precision: {round(precision_score(y_test, y_test_pred), 3)}")
print(f"recall: {round(recall_score(y_test, y_test_pred), 3)}")
print(f"f1 score: {round(f1_score(y_test, y_test_pred), 3)}")

accuracy: 0.628
precision: 0.588
recall: 0.539
f1 score: 0.563


In [10]:
# 2. 3-layer RNN using only the most correlated features (calculated above)
model1 = Sequential()
model1.add(Dense(150, input_dim=len(x_train1.columns), activation='relu'))
model1.add(Dense(75, activation='relu'))
model1.add(Dense(10, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model1.fit(x_train1, y_train1, epochs=80, batch_size=1024)

y_train_pred1 = model1.predict_classes(x_train1)
print(f"train accuracy (only using most correlated features): {round(accuracy_score(y_train1, y_train_pred1), 3)}")

y_test_pred1 = model1.predict_classes(x_test1)
print(f"test accuracy (only using most correlated features): {round(accuracy_score(y_test1, y_test_pred1), 3)}")
print(f"precision: {round(precision_score(y_test1, y_test_pred1), 3)}")
print(f"recall: {round(recall_score(y_test1, y_test_pred1), 3)}")
print(f"f1 score: {round(f1_score(y_test1, y_test_pred1), 3)}")

Epoch 1/80
67573/67573 [==============================] - 2s 30us/step - loss: 0.9173 - accuracy: 0.5735
Epoch 2/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6558 - accuracy: 0.6243
Epoch 3/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6511 - accuracy: 0.6275
Epoch 4/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6380 - accuracy: 0.6376
Epoch 5/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6373 - accuracy: 0.6366
Epoch 6/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6364 - accuracy: 0.6374
Epoch 7/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6349 - accuracy: 0.6396
Epoch 8/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6327 - accuracy: 0.6394
Epoch 9/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6349 - accuracy: 0.6401
Epoch 10/80
67573/67573 [==============================

67573/67573 [==============================] - 1s 10us/step - loss: 0.6189 - accuracy: 0.6540
Epoch 79/80
67573/67573 [==============================] - 1s 10us/step - loss: 0.6188 - accuracy: 0.6521
Epoch 80/80
67573/67573 [==============================] - 1s 10us/step - loss: 0.6184 - accuracy: 0.6523
train accuracy (only using most correlated features): 0.653
test accuracy (only using most correlated features): 0.654
precision: 0.637
recall: 0.52
f1 score: 0.573


In [16]:
# 3. 2-layer RNN using all of our features.
model2 = Sequential()
model2.add(Dense(1000, input_dim=len(x_train.columns), activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.fit(x_train, y_train, epochs=120, batch_size=1024)

Epoch 1/120
67573/67573 [==============================] - 4s 65us/step - loss: 2.9730 - accuracy: 0.5315
Epoch 2/120
67573/67573 [==============================] - 3s 50us/step - loss: 0.6723 - accuracy: 0.6000
Epoch 3/120
67573/67573 [==============================] - 3s 50us/step - loss: 0.6554 - accuracy: 0.6203
Epoch 4/120
67573/67573 [==============================] - 3s 46us/step - loss: 0.6557 - accuracy: 0.6226
Epoch 5/120
67573/67573 [==============================] - 3s 49us/step - loss: 0.6714 - accuracy: 0.6153
Epoch 6/120
67573/67573 [==============================] - 3s 46us/step - loss: 0.6491 - accuracy: 0.6274
Epoch 7/120
67573/67573 [==============================] - 3s 45us/step - loss: 0.6554 - accuracy: 0.6261
Epoch 8/120
67573/67573 [==============================] - 3s 45us/step - loss: 0.6443 - accuracy: 0.6336
Epoch 9/120
67573/67573 [==============================] - 4s 54us/step - loss: 0.6614 - accuracy: 0.6263
Epoch 10/120
67573/67573 [====================

In [17]:
y_test_pred2 = model2.predict_classes(x_test)
print(f"model 1 accuracy: {round(accuracy_score(y_test, y_test_pred2), 3)}")
print(f"precision: {round(precision_score(y_test, y_test_pred2), 3)}")
print(f"recall: {round(recall_score(y_test, y_test_pred2), 3)}")
print(f"f1 score: {round(f1_score(y_test, y_test_pred2), 3)}")

model 1 accuracy: 0.601
precision: 0.549
recall: 0.561
f1 score: 0.555
